In [ ]:
pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.6 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp

from gurobipy import*

mdl = Model("VRP")



'''
Sets and parameters
'''


N = [1, 2, 3, 4]

V = [0,1,2,3,4]

K = [1, 2]

A = [(i,j) for i in V for j in V if i!=j]

q = {1:6, 2:2, 3:4, 4:3}

C = {1:8, 2:4}

M = {1:1, 2:2}



'''
cost matrix
'''


c = {(0,1): 3,
(0, 2): 4,
(0, 3): 5,
(0, 4):2,
(1, 0):3,
(1, 2):5,
(1, 3):7,
(1, 4):5,
(2, 0):4,
(2, 1):5,
(2, 3):9,
(2, 4):6,
(3, 0):5,
(3, 1):7,
(3, 2):9,
(3, 4):4,
(4, 0):2,
(4, 1):5,
(4, 2):6,
(4, 3):4
}



'''
Decision Variable
'''

pairs= [(i,j,k) for i in V for j in V for k in K]

x = mdl.addVars(pairs, vtype=GRB.BINARY, name ="x")
mdl.modelSense = GRB.MINIMIZE



'''
Constraints
'''

mdl.addConstrs(quicksum(x[i,j,k] for k in K for i in V if i != j) == 1 for j in N)

mdl.addConstrs(quicksum(x[0,j,k] for j in N ) <= M[k] for k in K)

mdl.addConstrs((quicksum(x[i,j,k] for i in V) - quicksum(x[j,i,k] for i in V)) == 0 for j in V for k in K)

mdl.addConstrs(quicksum(x[i,j,k] for i in V for j in N if j != i) <= C[k] for k in K)

mdl.addConstrs(x[i,i,k] == 0 for i in V for k in K)



'''
Objective Function
'''

mdl.setObjective(quicksum(x[i]*c[arcs] for i in pairs for arcs in A))



'''
Solve
'''

mdl.Params.MIPGap = 0.1

mdl.Params.TimeLimit = 30  # seconds

mdl.optimize()

Restricted license - for non-production use only - expires 2024-10-28
Set parameter MIPGap to value 0.1
Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 28 rows, 50 columns and 162 nonzeros
Model fingerprint: 0x84ef677b
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 500.0000000
Presolve removed 12 rows and 10 columns
Presolve time: 0.00s
Presolved: 16 rows, 40 columns, 120 nonzeros
Variable types: 0 continuous, 40 integer (40 binary)

Root relaxation: objective 4.000000e+02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective B

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np


# Read input data from Excel file
input_df =
# Read distance matrix from file
distance_df =
# Read travel time matrix from file
traveltime_df =
# Read binary matrix from file
binary_df =

# Define the model
model = gp.Model('Last-mile Delivery and Pickup Routing Problem')

# Set of vehicles
V = range(1, 8)
#V = ['mo', 'tu', 'we', 'th', 'fr', 'sa', 'su']

# Set of customers
n = len(input_df) - 1  #Subtract 1 for column name #Depot is indexed in files as both 0 and n
C = input_df["Store_ID"].tolist()
M = 99999
Q = 500

# Start time for each customer
st = dict(zip(C, input_df["StartTime"].tolist()))

# End time for each customer
et = dict(zip(C, input_df["EndTime"].tolist()))

# Pickup demand of each customer
pu = dict(zip(C, input_df["Pickup"].tolist()))

# Delivery demand of each customer
de = dict(zip(C, input_df["Delivery"].tolist()))

# Distance between each pair of customers
d = {(i, j): distance_df.loc[i, j] for i in C for j in C}

# Travel time between each pair of customers
tv = {(i, j): traveltime_df.loc[i, j] for i in C for j in C}

# Binary matrix indicating if a customer can be visited by a particular type of vehicle
b = {(i, v): binary_df.loc[i, v] for i in C for v in V}

# Create decision variables
x = {}  # Binary variable for whether a route includes an arc
for v in V:
    for i in C:
        for j in C:
            x[i,j,v] = model.addVar(vtype=gp.GRB.BINARY, name=f"x_{i}_{j}_{v}")

y = {}  # Integer variable for load of products leaving depot
for v in V:
    y[0, v] = model.addVar(vtype=GRB.INTEGER, name=f"y_0_{v}")
z = {} # Integer variable for load of products after servicing j
for j in range(1,n+1):
    z[j] = model.addVar(vtype=gp.GRB.INTEGER, name=f"z_{j}")
t = {}  # Continuous variable for arrival time at the location
for v in V:
    for i in C:
        t[i, v] = model.addVar(vtype=gp.GRB.CONTINUOUS, name=f"t_{i}_{v}")


# Define objective function
model.setObjective(gp.quicksum(d[i, j] * x[i, j, v] for i in C for j in C if i !=j for v in V), GRB.MINIMIZE)

# Define constraints

# Each customer must be visited exactly once
for j in C:
            model.addConstr(gp.quicksum(x[i, j, v] for v in V for i in C if i != j) == 1, name='firstConst')

# Vehicle leaves node that it enters
for v in V:
    for h in range(1,n+1):
        model.addConstr(gp.quicksum(x[i,h,v] for i in C) == gp.quicksum(x[h,j,v] for j in C), name= '2nd_b')

# Each vehicle must start and end at the depot
for v in V:
    model.addConstr(gp.quicksum(x[0, j, v] for j in range(1, n+1)) == gp.quicksum(x[i, 0, v] for i in range(1, n+1)), name=f"depot_{v}")


for v in V:
        model.addConstr(y[0, v] == gp.quicksum(de[j]* x[i, j, v] for i in C for j in range(1,n+1) if j!=i), name='4Const')

for v in V:
    for j in range(1,n+1):
        model.addConstr(z[j] >= y[0, v] - de[j] + pu[j] - M* (1 - x[0, j, v]), name='5Const')

for i in range(1,n+1):
    for j in range(1,n+1):
        model.addConstr(z[j] >= z[i] - de[j] + pu[j] - M*(1 - gp.quicksum(x[i, j, v] for v in V)), name='6Const')

for v in V:
    model.addConstr(y[0,v] <= Q, name='7Const')

for v in V:
    for j in range(1,n+1):
        model.addConstr(z[j] <= Q + M*(1 - gp.quicksum(x[i, j, v] for i in C)), name='8Const')

for i in C:
    for j in C:
        model.addConstr(t[i,v] + 1/144 + tv[i, j] - M*(1 - gp.quicksum(x[i, j, v] for v in V)) <= t[j, v], name='9Const')

for i in C:
    for v in V:
        model.addConstr(t[i, v] >= st[i], name='10Const')
        model.addConstr(t[i, v] <= et[i], name='11Const')

model.setParam(GRB.Param.TimeLimit, 100.0)
# Solve the model
model.optimize()

if model.status == GRB.INFEASIBLE:
        model.computeIIS()
        model.write("infeasible.lp")
        print('\nThe following constraint(s) cannot be satisfied:')
        for c in model.getConstrs():
            if c.IISConstr:
                print('%s' % c.constrName)

# Check if the model is solved to optimality
if model.status == gp.GRB.OPTIMAL:
    # Print the optimal objective value
    print(f"Optimal objective value: {model.objVal}")
    # Print the solution for each variable
    for v in model.getVars():
        print(f"{v.varName} = {v.x}")

else:
    print("No solution found.")

FileNotFoundError: ignored